# Intro

This project is designed simulate a workflow where a logistics officer would use an AI to extract data from invoices and save them to a database.

In [1]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "🚀Custom_OpenAI_Agent"



In [2]:
# imports
import base64
import os
from enum import Enum
from io import BytesIO
from typing import Iterable
from typing import List
from typing import Literal, Optional

import fitz
# Instructor is powered by Pydantic, which is powered by type hints. Schema validation, prompting is controlled by type annotations
import instructor
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from PIL import Image
from openai import OpenAI
from pydantic import BaseModel, Field
import json


# Data Structures

In [3]:
# Define Pydantic models to extract data from sample_invoice
class InvoiceDetails(BaseModel):
    invoice_number: str = Field(..., title="Invoice Number")
    purchase_order_number: str = Field(..., title="Purchase Order")
    due_date: str = Field(..., title="Due Date")
    invoice_date: str = Field(..., title="Invoice Date")
    invoice_subtotal: str = Field(..., title="Invoice Subtotal")
    invoice_total: str = Field(..., title="Invoice Total")


## Helper functions

In [4]:
# Function to encode the image as base64
def encode_image(image_path: str):
    # check if the image exists
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    


def parse_invoice(base64_img: str) -> InvoiceDetails:
    response = instructor.from_openai(OpenAI()).chat.completions.create(
        model='gpt-4-turbo',
        response_model=InvoiceDetails,
        messages=[
            {
                "role": "user",
                "content": 'Analyze the given invoice and very carefully extract the information.',
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_img}"
                        }
                    },
                ],
            }
        ],
    )
    result = response.model_dump_json()
    return result


# Testing

In [ ]:
# create the base64 image
base64_image = encode_image("sample_invoice.png")

result = parse_invoice(base64_image)

print(result)



In [ ]:
# check if the result is a dictionary
if isinstance(result, dict):
    print("Result is a dictionary")
else:
    print("Result is not a dictionary")



In [ ]:
# Testing the type and content of the result
print(type(result))
print(result)

# If result is a JSON string, convert it to a dictionary
if isinstance(result, str):
    try:
        result = json.loads(result)
        print("Converted JSON string to dictionary.")
    except json.JSONDecodeError:
        print("Failed to decode JSON string.")

In [ ]:
# # Read in the saved JSON file
# with open("resume_data.json", "r") as file:
#     invoice_data = json.load(file)

# print(invoice_data)



# Simulating Data Saving

In [ ]:
# Simulate saving the data to the database
# This is where you would put your code to save the data to the database
# Start by creating a pandas dataframe from the json data
df = pd.DataFrame(result, index=[0])
# print the dataframe
print(df)






In [ ]:
df.head()

# Simulating saving to a database

In [5]:
# Create a sqlite database
import sqlite3

# Connect to a database (or create one if it doesn't exist)
conn = sqlite3.connect('invoices.db')

# Create a cursor object using the cursor method
cursor = conn.cursor()

# Create a table
cursor.execute('''
CREATE TABLE IF NOT EXISTS invoices (
    invoice_number TEXT,
    purchase_order_number TEXT,
    due_date TEXT,
    invoice_date TEXT,
    invoice_subtotal TEXT,
    invoice_total TEXT
)
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
# import sqlite3
# import pandas as pd

# # Connect to the SQLite database
# conn = sqlite3.connect('invoices.db')

# # Write the data to a sqlite table
# df.to_sql('invoices', conn, if_exists='append', index=False)

# # Commit the changes and close the connection
# conn.commit()
# conn.close()

In [ ]:
# import sqlite3

# # Connect to the SQLite database
# conn = sqlite3.connect('invoices.db')

# # Create a cursor object
# cursor = conn.cursor()

# # Execute a query to select all records from the 'invoices' table
# cursor.execute("SELECT * FROM invoices")

# # Fetch all rows from the query
# rows = cursor.fetchall()

# # Print each row
# for row in rows:
#     print(row)

# # Close the cursor and the connection
# cursor.close()
# conn.close()

In [ ]:
# import sqlite3
# import pandas as pd

# # Connect to the SQLite database
# conn = sqlite3.connect('invoices.db')

# # Query the database for all records in the 'invoices' table
# query = "SELECT * FROM invoices"
# df = pd.read_sql_query(query, conn)

# # Display the DataFrame
# print(df)

# # Close the connection
# conn.close()

# Simulating a complete workflow

In [6]:
def extract_invoice_data(input_image_path: str = "sample_invoice.png") -> InvoiceDetails:
    # Encode the image
    base64_img = encode_image(input_image_path)
    response = instructor.from_openai(OpenAI()).chat.completions.create(
        model='gpt-4-turbo',
        response_model=InvoiceDetails,
        messages=[
            {
                "role": "user",
                "content": 'Analyze the given invoice and very carefully extract the information.',
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_img}"
                        }
                    },
                ],
            }
        ],
    )
    extracted_data = response.model_dump_json()
    result = json.loads(extracted_data)
    return result



In [7]:
# A function to save results to a database
def save_results_to_database(results: dict):
    # Convert the results to a DataFrame
    df = pd.DataFrame(results, index=[0])
    # Connect to the SQLite database
    conn = sqlite3.connect('invoices.db')
    # Write the data to a sqlite table
    df.to_sql('invoices', conn, if_exists='append', index=False)
    # Commit the changes and close the connection
    conn.commit()
    conn.close()
    return "Data saved to database"



In [ ]:
invoice_data = extract_invoice_data()
save_results_to_database(invoice_data)



In [8]:
# Define a list of invoice image paths
invoices = ["sample_invoice.png", "sample_invoice_2.png"]

# Loop through each invoice in the list
for invoice_path in invoices:
    # Call the extract_invoice_data function for each invoice
    invoice_data = extract_invoice_data(invoice_path)
    # Optionally, print or process the extracted data
    print(f"Successfully extracted data from invoice: {invoice_path}")
    # save the data to the database
    save_results_to_database(invoice_data)
    print(f"Successfully saved data to database for invoice: {invoice_path}")



Successfully extracted data from invoice: sample_invoice.png
Successfully saved data to database for invoice: sample_invoice.png
Successfully extracted data from invoice: sample_invoice_2.png
Successfully saved data to database for invoice: sample_invoice_2.png


In [9]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('invoices.db')

# Query the database for all records in the 'invoices' table
query = "SELECT * FROM invoices"
df = pd.read_sql_query(query, conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()

  invoice_number purchase_order_number    due_date invoice_date  \
0         AU-001             1430/2019  26/04/2019   29/01/2019   
1         AU-001             1430/2019  26/04/2019   29/01/2019   

  invoice_subtotal invoice_total  
0           186.00        204.60  
1           186.00        204.60  
